<a href="https://colab.research.google.com/github/changedi/DPpro/blob/master/ArxivLoader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain --upgrade
# Version: 0.0.164

!pip install pypdf

!pip install python-dotenv

In [ ]:
# PDF Loaders. If unstructured gives you a hard time, try PyPDFLoader
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

In [ ]:
!pip install arxiv

In [ ]:
!pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 40.8 MB/s eta 0:00:00


In [ ]:
from langchain.retrievers import ArxivRetriever
retriever = ArxivRetriever(load_max_docs=2)
docs = retriever.get_relevant_documents(query="2208.13629")

In [ ]:
docs[0].metadata  # meta-information of the Document

{'Published': '2022-08-29',
 'Title': 'A Survey on Text-to-SQL Parsing: Concepts, Methods, and Future Directions',
 'Authors': 'Bowen Qin, Binyuan Hui, Lihan Wang, Min Yang, Jinyang Li, Binhua Li, Ruiying Geng, Rongyu Cao, Jian Sun, Luo Si, Fei Huang, Yongbin Li',
 'Summary': 'Text-to-SQL parsing is an essential and challenging task. The goal of\ntext-to-SQL parsing is to convert a natural language (NL) question to its\ncorresponding structured query language (SQL) based on the evidences provided\nby relational databases. Early text-to-SQL parsing systems from the database\ncommunity achieved a noticeable progress with the cost of heavy human\nengineering and user interactions with the systems. In recent years, deep\nneural networks have significantly advanced this task by neural generation\nmodels, which automatically learn a mapping function from an input NL question\nto an output SQL query. Subsequently, the large pre-trained language models\nhave taken the state-of-the-art of the t

In [ ]:
docs[0].page_content[:400]  # a content of the Document

'IEEE TRANSACTIONS ON KNOWLEDGE AND DATA ENGINEERING\n1\nA Survey on Text-to-SQL Parsing: Concepts,\nMethods, and Future Directions\nBowen Qin, Binyuan Hui, Lihan Wang, Min Yang, Jinyang Li, Binhua Li, Ruiying Geng, Rongyu Cao, Jian\nSun, Luo Si, Fei Huang, Yongbin Li\nAbstract—Text-to-SQL parsing is an essential and challenging task. The goal of text-to-SQL parsing is to convert a natural language\n(NL) '

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import os
from langchain.llms import OpenAI
from dotenv import load_dotenv

load_dotenv()
# for LangChain
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.6 MB/s eta 0:00:00


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

model = ChatOpenAI(model_name="gpt-3.5-turbo")  # switch to 'gpt-4'
qa = ConversationalRetrievalChain.from_llm(model, retriever=retriever)

In [ ]:
questions = [
    "What is Text-to-Sql?",
    "What is the main idea of this paper?",
    "What method does this paper provide?",
]
chat_history = []

for question in questions:
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result["answer"]))
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")

-> **Question**: What is Text-to-Sql? 

**Answer**: I don't know what "Text-to-Sql" refers to. 

-> **Question**: What is the main idea of this paper? 

**Answer**: The main idea of this paper is to analyze a bifurcation phenomenon in the theory of noise-activated transitions in double-well systems. The authors study the behavior of the most probable transition path between the two wells as a system parameter is varied. They demonstrate that at the bifurcation point, the activation kinetics of the system become non-Arrhenius, meaning that the growth of the mean time between inter-well fluctuations is not purely exponential. The authors propose a new scaling theory to explain the weak-noise behavior at the bifurcation point and show that it applies to a large universality class of double-well systems. 

-> **Question**: What method does this paper provide? 

**Answer**: This paper provides a method for systematically deriving a finite set of observables to construct a lifted linear stat

# OPENAI的api调用


## ChatCompletion

In [ ]:
# Example of an OpenAI ChatCompletion request
# https://platform.openai.com/docs/guides/chat
import time
import openai
# record the time before the request is sent
start_time = time.time()

# send a ChatCompletion request to count to 100
response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=[
        { 'role': 'user',
         'content': 'Count to 10, with a comma between each number and no newlines. E.g., 1, 2, 3, ...'
         }
    ],
    temperature=0,
)

# calculate the time it took to receive the response
response_time = time.time() - start_time

# print the time delay and text received
print(f"Full response received {response_time:.2f} seconds after request")
print(f"Full response received:\n{response}")

## PromptCompletion

In [ ]:
source = "A suffix between a prompt and a completion is necessary to tell the model that the input text has stopped, and that it now needs to predict the class. Since we use the same separator in each example, the model is able to learn that it is meant to predict either baseball or hockey following the separator. A whitespace prefix in completions is useful, as most word tokens are tokenized with a space prefix. The tool also recognized that this is likely a classification task, so it suggested to split the dataset into training and validation datasets. This will allow us to easily measure expected performance on new data."
prompt = f'''You are a helpful translater.
Translate this sentence {source} to Chinese.
'''
response = openai.Completion.create(
        prompt=prompt,
        engine='text-davinci-002',
        temperature=0,
        top_p=1,
        max_tokens=1500,
    )

print(f"Full response received:\n{response}")
print(response['choices'][0]['text'])

Full response received:
{
  "warning": "This model version is deprecated. Migrate before January 4, 2024 to avoid disruption of service. Learn more https://platform.openai.com/docs/deprecations",
  "id": "cmpl-80o7i2IcpIQHe0Z5pQnQU6oQ0UkOf",
  "object": "text_completion",
  "created": 1695203366,
  "model": "text-davinci-002",
  "choices": [
    {
      "text": "\n\u4e00\u4e2a\u540e\u7f00\u5728\u63d0\u793a\u548c\u5b8c\u6210\u4e4b\u95f4\u662f\u5fc5\u8981\u7684\uff0c\u544a\u8bc9\u6a21\u578b\u8f93\u5165\u6587\u672c\u5df2\u7ecf\u505c\u6b62\uff0c\u73b0\u5728\u9700\u8981\u9884\u6d4b\u7c7b\u522b\u3002\u7531\u4e8e\u6211\u4eec\u5728\u6bcf\u4e2a\u793a\u4f8b\u4e2d\u4f7f\u7528\u76f8\u540c\u7684\u5206\u9694\u7b26\uff0c\u6a21\u578b\u80fd\u591f\u5b66\u4e60\u5230\u5b83\u9700\u8981\u5728\u5206\u9694\u7b26\u4e4b\u540e\u9884\u6d4b\u68d2\u7403\u6216\u51b0\u7403\u3002\u5728\u5b8c\u6210\u4e2d\u4f7f\u7528\u7a7a\u683c\u524d\u7f00\u662f\u6709\u7528\u7684\uff0c\u56e0\u4e3a\u5927\u591a\u6570\u5355\u8bcd\u4ee4\u7

## Embedding

In [ ]:
from tenacity import retry, wait_random_exponential, stop_after_attempt, retry_if_not_exception_type
EMBEDDING_MODEL = 'text-embedding-ada-002'
EMBEDDING_CTX_LENGTH = 8191
EMBEDDING_ENCODING = 'cl100k_base'

# let's make sure to not retry on an invalid request, because that is what we want to demonstrate
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6), retry=retry_if_not_exception_type(openai.InvalidRequestError))
def get_embedding(text_or_tokens, model=EMBEDDING_MODEL):
    return openai.Embedding.create(input=text_or_tokens, model=model)["data"][0]["embedding"]

long_text = 'AGI ' * 2000
try:
    embedding=get_embedding(long_text)
    print(len(embedding))
    print(embedding[:10])
except openai.InvalidRequestError as e:
    print(e)

1536
[-0.013576747849583626, 0.005601828917860985, -0.008355950005352497, -0.016150379553437233, -0.00665802089497447, 0.0058759041130542755, -0.017634397372603416, 0.00013150175800547004, -0.005264248698949814, -0.024145353585481644]


## Model list

In [ ]:
modelList = openai.Model.list()
modelTypes = set()
for model in modelList['data']:
  modelTypes.add(model['owned_by'])
print(modelTypes)

for tp in modelTypes:
  print(f"模型类型:{tp}")
  for model in modelList['data']:
    if(model['owned_by']==tp and model['object']=='model'):
      print(f"\t{model['id']}")

{'system', 'openai-internal', 'user-pdd55umvas6wjtxw8bwroyn0', 'openai', 'openai-dev'}
模型类型:system
	babbage-002
	davinci-002
	gpt-3.5-turbo-instruct-0914
	gpt-3.5-turbo-instruct
模型类型:openai-internal
	text-davinci-003
	text-embedding-ada-002
	whisper-1
	gpt-3.5-turbo-16k
模型类型:user-pdd55umvas6wjtxw8bwroyn0
	curie:ft-personal-2023-08-18-07-44-43
	ft:gpt-3.5-turbo-0613:personal:recipe-ner:7zxt7MdW
	ada:ft-personal-2023-08-17-07-46-18
模型类型:openai
	babbage
	text-babbage-001
	davinci
	code-davinci-edit-001
	curie-instruct-beta
	davinci-instruct-beta
	text-curie-001
	text-davinci-001
	gpt-3.5-turbo-16k-0613
	text-davinci-002
	text-davinci-edit-001
	gpt-3.5-turbo-0613
	ada
	text-ada-001
	gpt-3.5-turbo-0301
	curie
	gpt-3.5-turbo
模型类型:openai-dev
	curie-search-query
	babbage-search-document
	text-search-babbage-doc-001
	text-similarity-davinci-001
	davinci-similarity
	curie-similarity
	code-search-babbage-code-001
	text-search-ada-doc-001
	text-search-davinci-doc-001
	davinci-search-query
	text-si

## DALL-E image

In [ ]:
import requests
from PIL import Image
# set a directory to save DALL·E images to
image_dir_name = "images"
image_dir = os.path.join(os.curdir, image_dir_name)

# create the directory if it doesn't yet exist
if not os.path.isdir(image_dir):
    os.mkdir(image_dir)

# print the directory to save to
print(f"{image_dir=}")
# create an image

# set the prompt
# prompt = "A cyberpunk monkey hacker dreaming of a beautiful bunch of bananas, digital art"
prompt = "a boy, Happy, Chest shot, "

# call the OpenAI API
generation_response = openai.Image.create(
    prompt=prompt,
    n=1,
    size="512x512",
    response_format="url",
)

# print response
print(generation_response)
# save the image
generated_image_name = "generated_image.png"  # any name you like; the filetype should be .png
generated_image_filepath = os.path.join(image_dir, generated_image_name)
generated_image_url = generation_response["data"][0]["url"]  # extract image URL from response
generated_image = requests.get(generated_image_url).content  # download the image

with open(generated_image_filepath, "wb") as image_file:
    image_file.write(generated_image)  # write the image to the file
display(Image.open(generated_image_filepath))

{
  "created": 1695203548,
  "data": [
    {
      "url": "https://oaidalleapiprodscus.blob.core.windows.net/private/org-AC7zlmDTl6XMyqvd7y7LmsOp/user-PdD55uMvaS6WJTxW8BwrOyn0/img-MfKAc3lmUIcUkv1fnrUVLE3J.png?st=2023-09-20T08%3A52%3A28Z&se=2023-09-20T10%3A52%3A28Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2023-09-20T00%3A07%3A17Z&ske=2023-09-21T00%3A07%3A17Z&sks=b&skv=2021-08-06&sig=oZFEQY22HCoPnOJvU5/af%2B2ehsTlnzWFofg%2Bg0FGPEI%3D"
    }
  ]
}


NameError: ignored